In [1]:
# dpo_idefics2-8b.py
from datasets import features, load_dataset
from transformers import AutoModelForVision2Seq, AutoProcessor
import torch
from trl import DPOConfig, DPOTrainer
from peft import LoraConfig

import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/home/users1/knupleun/home/projects/TARUN_TEST/venv/lib64/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/users1/knupleun/home/projects/TARUN_TEST/venv/lib64/python3.10/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
No ROCm runtime is found, using ROCM_HOME='/usr'


In [5]:
# Load the model and processor
model = AutoModelForVision2Seq.from_pretrained(
    "llava-hf/llava-v1.6-mistral-7b-hf",
    torch_dtype=torch.float16,
    cache_dir="./cache",
    load_in_4bit=True,
)
processor = AutoProcessor.from_pretrained(
    "llava-hf/llava-v1.6-mistral-7b-hf", do_image_splitting=False, cache_dir="./cache"
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now default to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.10s/it]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [6]:
# Load the dataset
dataset = load_dataset(
    "openbmb/RLAIF-V-Dataset", split="train[:500]", cache_dir="cache"
)


def format(example):
    # Prepare the input for the chat template
    prompt = [
        {
            "role": "user",
            "content": [
                {"type": "image"},
                {"type": "text", "text": example["question"]},
            ],
        }
    ]
    chosen = [
        {"role": "assistant", "content": [{"type": "text", "text": example["chosen"]}]}
    ]
    rejected = [
        {
            "role": "assistant",
            "content": [{"type": "text", "text": example["rejected"]}],
        }
    ]
    # Apply the chat template
    prompt = processor.apply_chat_template(prompt, tokenize=False)
    chosen = processor.apply_chat_template(chosen, tokenize=False)
    rejected = processor.apply_chat_template(rejected, tokenize=False)
    # Resize the image to ensure it fits within the maximum allowable
    # size of the processor to prevent OOM errors.
    # max_size = processor.image_processor.size["longest_edge"] // 2
    # example["image"].thumbnail((max_size, max_size))
    return {
        "images": [example["image"]],
        "prompt": prompt,
        "chosen": chosen,
        "rejected": rejected,
    }


# Apply the formatting function to the dataset
dataset = dataset.map(format, remove_columns=dataset.column_names, num_proc=32)

# Make sure that the images are decoded, it prevents from storing bytes.
# More info here https://github.com/huggingface/blog/pull/2148#discussion_r1667400478
f = dataset.features
f["images"] = features.Sequence(features.Image(decode=True))
dataset = dataset.cast(f)

In [7]:
# Train the model
training_args = DPOConfig(
    output_dir="idefics2-8b-dpo",
    bf16=False,
    gradient_checkpointing=True,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    dataset_num_proc=8,  # tokenization will use 32 processes
    dataloader_num_workers=32,  # data loading will use 32 workers
    logging_steps=10,
)

In [8]:
trainer = DPOTrainer(
    model,
    ref_model=None,  # not needed when using peft
    args=training_args,
    train_dataset=dataset,
    processing_class=processor,
    peft_config=LoraConfig(target_modules="all-linear", r=32),
)

Extracting prompt from train dataset (num_proc=8): 100%|██████████| 500/500 [00:31<00:00, 15.95 examples/s]
Applying chat template to train dataset (num_proc=8): 100%|██████████| 500/500 [00:00<00:00, 698.58 examples/s] 
Tokenizing train dataset (num_proc=8): 100%|██████████| 500/500 [00:08<00:00, 61.89 examples/s] 


In [ ]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/home/users1/knupleun/home/projects/TARUN_TEST/venv/lib64/python3.10/site-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss
